# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_pd = pd.read_csv("../output_data/hw_cities_output.csv")
cities_pd.head()


,City,lats,lngs,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,castro,-24.7911,-50.0119,61.52,68,47,8.68,BR,1624470078
1,santa rosa,14.3122,121.1114,80.58,86,100,4.45,PH,1624469545
2,barrow,71.2906,-156.7887,44.62,76,1,6.91,US,1624470001
3,ulaangom,49.9811,92.0667,54.68,50,27,6.46,MN,1624470078
4,enshi,30.3000,109.4833,69.12,86,35,0.76,CN,1624470079


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
figure_layout ={
    'width': '1200px',
    'height': '500px',
        'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

locations = cities_pd[['lats','lngs']]
rating = cities_pd['Humidity'].astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                 dissipating=False, max_intensity=0,
                                 point_radius=5)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# My perfect Vacation
# Countries in the Soutern Hemisphere, Temps between 80-90, Wind below 15
Vacation_df = cities_pd.loc[(cities_pd['lats'] < 0) & (cities_pd['Wind Speed'] < 15) &
                            (cities_pd['Max Temp'] > 80) & (cities_pd['Max Temp'] < 90),:]
                           
Vacation_df.count()
Vacation_df.head(50)

,City,lats,lngs,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,campoverde,-8.4753,-74.8058,85.69,43,76,1.41,PE,1624470084
34,lagunas,-5.2269,-75.6753,88.84,52,9,1.12,PE,1624470089
80,hithadhoo,-0.6000,73.0833,82.60,71,69,9.55,MV,1624470105
94,kavieng,-2.5744,150.7967,81.59,79,49,9.17,PG,1624470109
146,tual,-5.6667,132.7500,81.46,76,32,6.24,ID,1624470127
205,faanui,-16.4833,-151.7500,80.15,75,16,12.10,PF,1624470148
212,maracacume,-2.0428,-45.9592,85.91,62,24,3.60,BR,1624470151
241,soure,-0.7167,-48.5233,83.17,69,18,6.55,BR,1624470161
269,sao gabriel da cachoeira,-0.1303,-67.0892,84.00,70,40,4.61,BR,1624470171
312,aquiraz,-3.9014,-38.3911,86.09,67,33,8.70,BR,1624470185


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
hotel_df = Vacation_df
hotel_df['hotel name'] = pd.Series(dtype='str')
hotel_df
params= {
    'radius': 5000
    'types': 'hotel',
    'key': g_key
}

for index, row in hotel_df.iterrows():
    lat = row['lats']
    lng = row['lngs']

    params['location'] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)

    name_address = name_address.json()

    try:
        hotel_df.loc[index, "hotel name"] = name_address['results'][0]['name']
        hotel_df.loc[index, "hotel name"] = name_address['results'][0]['vicinity']
        hotel_df.loc[index, "hotel name"] = name_address['results'][0]['rating']
    except:
        print("No hotels in this remote part of the world...")


C:\Users\GPSchool\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
No hotels in this remote part of the world...
No hotels in this remote part of the world...
No hotels in this remote part of the world...
No hotels in this remote part of the world...
No hotels in this remote part of the world...
No hotels in this remote part of the world...
No hotels in this remote part of the world...
No hotels in this remote part of the world...
No hotels in this remote part of the world...
No hotels in this remote part of the world...
No hotels in this remote part of the world...
No hotels in this remote part of the world...
No hotels in this remote part of the world...
No hotels in this remote

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
